In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df=pd.read_csv('dataset.csv')
df

In [ ]:
lotss=df['SystemCodeNumber'].unique()
lots=lotss[0:14]    #making a list of parking lots to get plots one by one
lotss

In [ ]:
df["TrafficCondition_Code"] = df["TrafficConditionNearby"].astype("category").cat.codes  # low:2,high:1,avg:0
df["vehicle_Code"] = df["VehicleType"].astype("category").cat.codes  #car:1,bike:0,truck:3:cycle:2
df

In [ ]:
plots=[]

for lot in lots:
  import datetime
  df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],   #combining date and time columns and converting it in datetime format
                                  format='%d-%m-%Y %H:%M:%S')
  df_lot=df[df['SystemCodeNumber']==lot]   #making dataframe for one lot at a time
  df_lot = df_lot.sort_values('Timestamp').reset_index(drop=True) #sorting time
  class ParkingSchema(pw.Schema):
    Capacity:int
    Occupancy:int
    Timestamp:str
    SystemCodeNumber:str
    VehicleType:str
    TrafficConditionNearby:str
    QueueLength:int
    IsSpecialDay:int
    TrafficCondition_Code:int
    vehicle_Code:int
  filename = f"parking_stream_{lot}.csv"
  df_lot[["Timestamp", "Occupancy", "Capacity", "SystemCodeNumber","VehicleType","TrafficConditionNearby","QueueLength","IsSpecialDay","TrafficCondition_Code","vehicle_Code"]].to_csv(filename, index=False)
  data = pw.demo.replay_csv(filename, schema=ParkingSchema, input_rate=1000)   # Load the data as a simulated stream using Pathway's replay_csv function
  fmt = "%Y-%m-%d %H:%M:%S"
  data_with_time = data.with_columns(
      t = data.Timestamp.dt.strptime(fmt),   #containes full datetime
      day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),  #contains only day date
      hour = data.Timestamp.dt.strptime(fmt).dt.hour(),   #contains hour
      day_of_week = data.Timestamp.dt.strptime(fmt).dt.weekday(),   #assigns monday: 0,...,sunday: 6
      occupancy_rate = data.Occupancy / data.Capacity
  )
  def time_of_day_weight(hour):   #the reason for choosing these categories is explained in the report

    if 11 <= hour < 14:
        return 1.0    # Midday
    elif 14 <= hour < 17:
        return 0.7    # Evening
    else:
        return 0.4    # Morning

  def weekday_weight(day_of_week):

    if day_of_week < 5:
        return 1.0    # Weekday
    else:
        return 0.7    # Weekend
  def queue_time_of_day_weight(hour):

    if 12<= hour < 14:
        return 1.0
    else:
        return 0.7
  def traffic_time_of_day_weight(hour):

    if 12<= hour < 14:
        return 1.0

    else:
        return 0.5

  def hour_effect(hour):
    a = 1.0
    mu = 12
    sigma = 3
    return a * np.exp(-((hour - mu) ** 2) / (2 * sigma ** 2))
  beta=1.0
  gamma=1.0
  delta=1.0
  epsilon=1.0
  zeta=1.0
  vehicle_weights = {'car': 1.0, 'bike': 0.8, 'truck': 1.3, 'cycle': 0.7}
  def demand_fn(occ_rate,tod_weight,wd_weight,adj_queue,adj_traffic,hour_term,veh_term,spec_term):    #demand fn
    return (
       2.0 * occ_rate * tod_weight * wd_weight+beta * np.log1p(adj_queue)+gamma * np.log1p(adj_traffic) + delta * hour_term + epsilon * veh_term + zeta * spec_term
    )
  import datetime
  data_with_price=(
     data_with_time.with_columns(

         tod_weight = pw.apply(time_of_day_weight, data_with_time.hour),
         wd_weight = pw.apply(weekday_weight, data_with_time.day_of_week),
         queue_weight = pw.apply(queue_time_of_day_weight,data_with_time.hour),
         traffic_weight = pw.apply(traffic_time_of_day_weight,data_with_time.hour),
     )
     .with_columns(

         adj_queue = pw.apply(lambda q, w: q * w, data_with_time.QueueLength, pw.this.queue_weight),
         adj_traffic = pw.apply(lambda t, w: t * w, data_with_time.TrafficCondition_Code, pw.this.traffic_weight),

         hour_term = pw.apply(hour_effect,data_with_time.hour),
         veh_term = vehicle_weights.get(data_with_time.VehicleType, 1.0),
         spec_term = data_with_time.IsSpecialDay
     )
     .with_columns(
      demand=pw.apply(demand_fn,
                      data_with_time.occupancy_rate,
                      pw.this.tod_weight,
                       pw.this.wd_weight,
                      pw.this.adj_queue,
                      pw.this.adj_traffic,
                      pw.this.hour_term,
                      pw.this.veh_term,
                      pw.this.spec_term
      )
     )
  )
  LAMBDA=1.0






  def normalize(d):
    return (d - 2) / (8 - 2)
  def price_fn(demand_norm, LAMBDA=LAMBDA):
    return 10 * (1 + LAMBDA * demand_norm)
 # aligned_data = data_with_price.demand.with_universe_of(data_with_price)
  plot_table=(data_with_price.with_columns(
     # with_price=data_with_price
      demand_norm = pw.apply(normalize,data_with_price.demand)
  )
  .with_columns(
      price = pw.apply(price_fn, pw.this.demand_norm)
  )
  )
  def price_plotter(source):  #writing bokeh plot fn

    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title=f"Pathway: Daily Parking Price of  {lot}",
        x_axis_type="datetime",
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.scatter("t", "price", source=source, size=6, color="red")

    return fig


  viz = plot_table.plot(price_plotter, sorting_col="t")
  plots.append(pn.Column(f"Lot: {lot}", viz.servable()))
dashboard = pn.Column(*plots)
dashboard.servable()





In [ ]:
pw.run()